# Two_tone_fit

## Imports

In [1]:
# MainScript.py

import numpy as np
import matplotlib.pyplot as plt

from labellines import *
import time

from scqubits import *
import scqubits
scqubits.settings.NUM_CPUS = 4
scqubits.settings.OVERLAP_THRESHOLD  = 1.0 #  In order to look for the indices


from scipy.optimize import minimize

%matplotlib qt
from tqdm import tqdm



from labmate.acquisition_notebook import AcquisitionAnalysisManager


from Multi_mode_aux import *

## Plotting

### Importing the file

In [2]:
# Define paths to files and other necessary variables
data_directory = r'S:\__ExperimentalData\Ouroboros\Data\Labmate'
file_paths = [
    r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_11__19_58_37__Two tones with ext. microwave src.h5",
    r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_14__10_56_45__Two tones with ext. microwave src.h5",
    # r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_15__17_16_51__Two tones with ext. microwave src.h5",
    # r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_15__00_35_05__Two tones with ext. microwave src.h5",
    # r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_16__12_51_19__Two tones with ext. microwave src.h5",
    # r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_16__13_35_56__Two tones with ext. microwave src.h5",
    # r"S:\__ExperimentalData\Ouroboros\Data\Labmate\Two tones with ext. microwave src\2024_03_17__11_59_53__Two tones with ext. microwave src.h5"
]



# Load and process data
processed_data = load_and_process_data(data_directory, file_paths)

### Transparency and scale_bar of the plots

In [3]:
# Define alpha, vmin, and vmax values for each plot
alphas = [1.0, 0.5]  # Example alpha values for each plot
vmin_values = [-0.0005, 0.00]  # Example vmin values for each plot
vmax_values = [0.0005, 0.0]  # Example vmax values for each plot
auto_files = [1]  # Indices of files where you want to use 'auto' for vmin and vmax

### Plotting the data

In [4]:
 # Plot the processed data
plot_data(processed_data, alphas, vmin_values, vmax_values, auto_files)

# Plotting the eigen energies

In [7]:
# Plot theory energies, this plots the eigen energies

plt.close()
fig = plt.figure(num = 2) # empty dataset
p0 = {'Ej':6.98e9,'Ec':0.879e9, 'El':0.054e9, 'g1':2.106e9, 'g2':1.153e9,'g3':0.1e9, 'w1':3.8e9,'w2':6.1e9,'w3':6.87e9}  
plot_theory_energies(fig, p0, flux_values=np.linspace(-1.1, 0.1, 10), levels=["1","2","3","4","5","6"])

# Plotting the transitions

In [5]:
## Define integer_flux, amp per phi0 and number of oscillators
Amp_per_phi0 = 0.1429013 
current_integer_flux = 0.02724955
n_oscillators = 3

In [10]:
# Plot theory lines

fig = plt.figure(num = 1) # empty dataset
# Define your parameters dictionary
p0 = {'Ej':6.98e9,'Ec':0.879e9, 'El':0.054e9, 'g1':2.106e9, 'g2':1.153e9,'g3':0.1e9, 'w1':3.8e9,'w2':6.1e9,'w3':6.87e9}  

plot_theory_lines(fig, p0, flux_values=np.linspace(-0.54, -0.4356, 10), transitions=["15","05","04","03","14","13","01","16","06","17","07","18","08","19","09"])

# Fit

In [5]:
# to remove the theory lines
for i in range(10):
    remove_lines(fig)

In [5]:
# Plot extracted points
extracted_points = {
    -0.042593:{
                '05':3.0338e9,
                '15':3.0050e9,
                '04':2.9858e9,
                '14':2.9566e9,
            },
        
    -0.043276:{
                '05':3.0242e9,
                '15':3.0079e9,
                '04':2.9818e9,
                '14':2.9665e9,
            },
    -0.044202:{ 
                '01':2.6e7,
                '16':5.765e9,
                '06':5.765e9,
                '17':5.809e9,
                '07':5.809e9, 
                '05':3.0123e9,
                '15':3.0123e9,
                '04':2.9778e9,
                '14':2.9778e9,
                # '113':6.8673e9,
                # '018':8.9727e9,
                # '024':9.7571e9,
                 # '112':6.8632e9,
            },

     # 0.026434:{
     #            '01':1.2814e9,
     #            '02':1.3062e9,
     #        },
     0.025413:{
                '01':1.2634e9,
                '02':1.3262e9,
                },

    # -0.0442011:{
    #             '01':4.6e7
    #            },

#          -0.06556:{
#                 '03':2.9575e9,
                
#             },

#          -0.066854:{
#                 '03':2.9737e9,
               
#             },

#           -0.067705:{
#                 '03':2.98e9,
               
#             },
 }

In [6]:
plot_extracted_points(fig,extracted_points)

In [7]:
# Perform minimization
p0 = {'Ej': 6.98e9, 'Ec': 0.879e9, 'El': 0.054e9, 'g1': 2.106e9, 'g2': 1.153e9, 'g3': 0.1e9, 'w1': 3.8e9, 'w2': 6.1e9, 'w3': 6.87e9}

In [8]:
def function_to_minimize(parameters_list, extracted_points):
    unrescaled_parameters = unrescale_parameter_array(parameters_list)
    parameters = from_parameter_array_to_parameter_dict(unrescaled_parameters, n_oscillators)
    return cost_function(parameters=parameters, extracted_points=extracted_points, model=model, cost_type='absolute', cutoff=70)

# Perform minimization
minimization = minimize(function_to_minimize, x0=rescale_parameter_array(list(p0.values())), args=(extracted_points,), method='Nelder-Mead')

In [9]:
from_parameter_array_to_parameter_dict(unrescale_parameter_array(minimization.x), n_oscillators)

{'Ej': 7163497855.641307,
 'Ec': 850761615.2939371,
 'El': 59269783.69410215,
 'g1': 2241888103.4561534,
 'g2': 1129848831.1574388,
 'g3': 97916103.62846233,
 'w1': 3468160311.591037,
 'w2': 6139475596.244926,
 'w3': 6927285150.353502}

In [10]:
for i in range(10):
    remove_lines(fig)
plot_extracted_points(fig, extracted_points)

In [11]:
plot_theory_lines(fig, from_parameter_array_to_parameter_dict(unrescale_parameter_array(minimization.x), n_oscillators), flux_values=np.linspace(-1.1, 0.1,500), transitions=['01', '02', '03', '04', '05','06','07','08','09','15','14','13','16','17','18','19','018','019','020', '021', '022', '023', '024', '025', '026','027',
'028','029','110', '111', '112', '113', '114', '115', '116', '117', '118', '119'])